In [1]:
from collections import Counter

# assigning 1 to player X, -1 to player O and 0 to empty cell
BOARD_EMPTY = 0
BOARD_PLAYER_X = 1
BOARD_PLAYER_O = -1

# determines whose turn it is (player X or player O) based on current state of the tic-tac-toe board
def player(s):
    counter = Counter(s)
    x_places = counter[1]
    o_places = counter[-1]

    if x_places + o_places == 9:
        return None
    elif x_places > o_places:
        return BOARD_PLAYER_O
    else:
        return BOARD_PLAYER_X

In [2]:
# determines 'player whose turn it is' and 'possible moves' based on the current game state
def actions(s):
    play = player(s)
    actions_list = [(play, i) for i in range(len(s)) if s[i] == BOARD_EMPTY]
    return actions_list

# determines new game state after the player has made the move
def result(s, a):
    (play, index) = a
    s_copy = s.copy()
    s_copy[index] = play
    return s_copy

# returns: "1" or "-1" for winner, 0 for tie and 'None' if game is not over yet
def terminal(s):
    for i in range(3):
        if s[3 * i] == s[3 * i + 1] == s[3 * i + 2] != BOARD_EMPTY:
            return s[3 * i]
        if s[i] == s[i + 3] == s[i + 6] != BOARD_EMPTY:
            return s[i]

    if s[0] == s[4] == s[8] != BOARD_EMPTY:
        return s[0]
    if s[2] == s[4] == s[6] != BOARD_EMPTY:
        return s[2]

    if player(s) is None:
        return 0

    return None

In [3]:
# importing required packages
import random
from math import log, sqrt

In [4]:
# random monte carlo function (without any playout policy)
def monte_carlo_random(s, simulations=2000):
    move = [] # initializing move list; this will save the first child node for every simulation
    score_per_sim = [] # initializing score_per_sim list; this will save the score (1 if agent wins, -1 if loses and 0 if tie) for every simulation

    for _ in range(simulations): # iteratively running simulations to a terminal state and saving the move with its respective score for each simulation
      s_copy = s.copy()
      move_per_sim = []
      available_moves = actions(s_copy)
      random_move = random.choice(available_moves)
      s_copy = result(s_copy, random_move)
      while terminal(s_copy) is None:
        available_moves = actions(s_copy)
        random_move = random.choice(available_moves)
        s_copy = result(s_copy, random_move)
      move.append(random_move[1])

      if terminal(s_copy) == BOARD_PLAYER_O:
        score_per_sim.append(1)
      elif terminal(s_copy) == BOARD_PLAYER_X:
        score_per_sim.append(-1)
      else:
        score_per_sim.append(0)

    # selecting the best move based on the score (selecting the move with the highest score)
    x = list(zip(move, score_per_sim))
    result_dict = {}
    for item in x:
      key, value = item
      if key not in result_dict:
        result_dict[key] = {'sum': 0, 'count': 0}
      result_dict[key]['sum'] += value
      result_dict[key]['count'] += 1
    print(result_dict)

    res = [(key, value['sum'] / value['count']) for key, value in result_dict.items()]
    action = max(res, key=lambda l: l[1])
    action = (-1,)+ action[:1]
    return action

In [5]:
# setting this up to build Monte Carlo with UCT policy

class Node:
  def __init__(self, state, parent):
    self.state = state
    self.parent = parent
    self.children = []
    self.wins = 0
    self.visits = 0

  # determines the child with the highest UCB score
  def select(self, c=1.4):
    if not self.children:
      return self
    max_ucb = -float('inf')
    best_child = None
    for child in self.children:
      if child.visits != 0:
        ucb = child.wins / child.visits + c * sqrt(log(self.visits) / child.visits)
      else:
        ucb = float('inf')

      if ucb > max_ucb:
        max_ucb = ucb
        best_child = child
    return best_child

  # expands the node and adds all possible child nodes to children
  def expand(self):
    for action in actions(self.state):
      new_state = result(self.state, action)
      child = Node(new_state, self)
      self.children.append(child)

  # simulates a game and returns final result (1 if agent wins, -1 if loses and 0 if tie)
  def simulate(self):
    state = self.state
    while terminal(state) is None:
      available_moves = actions(state)
      random_move = random.choice(available_moves)
      state = result(state, random_move)
    return -terminal(state)

  # backpropagates the results (wins and visits) for each visited node during the simulations
  def backpropagate(self, result_per_sim):
    self.wins += result_per_sim
    self.visits += 1
    if self.parent is not None:
      self.parent.backpropagate(result_per_sim)


In [6]:
# building Monte Carlo with UCT function to run simulations and return the best next move
def monte_carlo_UCT(root, simulations=2000):
  for _ in range(simulations):
    node = root.select()
    node.expand()
    result_per_sim = node.simulate()
    node.backpropagate(result_per_sim)
  return root.select()

In [7]:
# to print the board in a nicer format
def print_board(s):
    def convert(num):
        if num == BOARD_PLAYER_X:
            return 'X'
        if num == BOARD_PLAYER_O:
            return 'O'
        return '_'

    i = 0
    for _ in range(3):
        for _ in range(3):
            print(convert(s[i]), end=' ')
            i += 1
        print()

In [ ]:
# function to play the tic-tac-toe game
if __name__ == '__main__':
    s = [BOARD_EMPTY for _ in range(9)] # setting initial board with all 0's
    print('|------- WELCOME TO TIC TAC TOE -----------|')
    print('You are X while the Computer is O')

    while terminal(s) is None: # to play the game till someone wins or it ties
        play = player(s)
        if play == BOARD_PLAYER_X:
            print('\n\nIt is your turn', end='\n\n')
            x = int(input('Enter the x-coordinate [0-2]: '))
            y = int(input('Enter the y-coordinate [0-2]: '))
            index = 3 * x + y

            if not s[index] == BOARD_EMPTY:
                print('That coordinate is already taken. Please try again.')
                continue

            s = result(s, (1, index))
            print_board(s)
        else:
            print('\n\nThe computer is playing its turn')
            root = Node(s, None)                      # uncomment for monte_carlo_UCT and comment for monte_carlo_random
            bestchildnode = monte_carlo_UCT(root)     # uncomment for monte_carlo_UCT and comment for monte_carlo_random
            s = bestchildnode.state                   # uncomment for monte_carlo_UCT and comment for monte_carlo_random
            #action = monte_carlo_random(s)           # uncomment for monte_carlo_random and comment for monte_carlo_UCT
            #s = result(s, action)                    # uncomment for monte_carlo_random and comment for monte_carlo_UCT
            print_board(s)

    # determining and printing the winner
    winner = terminal(s)
    if winner == BOARD_PLAYER_X:
        print("You have won!")
    elif winner == BOARD_PLAYER_O:
        print("You have lost!")
    else:
        print("It's a tie.")